
### step 1: make list of elements of file

        list_img = os.listdir(file_addr)

### step 2: resize all images from input folder and store in input_resize folder

        for i in list_img:
            im = Image.open(input_file_addr+'\\'+i)
            im1 = im.resize((200,200))
            im2 = im1.convert('RGB')
            im2.save(input_resize_file_addr+'\\'+i,'JPEG')   
    
### step 3: flatten images to matrix(m,n_x)

        X = np.array([np.array(Image.open(input_file_addr+'\\'+i)).flatten() for i in list_img],'f')
        X = X.T

### step 4: labelling the dataset

        m = X.shape[1]           #no. of images
        m_t = X_t.shape[1]
        Y = np.zeros((m,1),dtype=int)
        Y_t = np.zeros((m_t,1),dtype=int)
        Y[0:m] = 1 

### step 5: reshape Y to maintain the consistency

        Y = Y.reshape((1,X.shape[1])).T

### step 6: shuffle data (need to do it for better result)

        X_train,Y_train = shuffle(X,Y, random_state=0)

### step 7: standardize the data (not neccessary but good practice)

        X_train = X_train/255   #255 is maximum possible value in image pixle 

### step 8: do all above steps for test data

        X_test = ....
        Y_test = ....

### step 9: run the model function

        n_h1 = 7      #number of nodes in layer
        d = model(X_train, Y_train, X_test, Y_test,n_h1, num_iterations = 6000, learning_rate = 0.05,lambd = 0)

### step 10: Print train/test Errors

        Y_prediction_train = d["Y_prediction_train"]
        Y_prediction_test = d["Y_prediction_test"]

        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

### step 11:draw cost vs iteration graph
    
        plot_cost(cost)

In [2]:
from PIL import Image
import numpy as np
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import *

In [40]:
#sigmoid function
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [41]:
#initialization of parameter w,b
def initialize(n_x):
    W = np.random.randn(1,n_x)*0.01
    #W = np.zeros(shape=(1,n_x))
    b = 0
    assert(W.shape == (1,n_x))
    assert(isinstance(b, float) or isinstance(b, int))

    return W,b

In [42]:
#forword and backword propagation
def propagate(X,Y,W,b):
    m = X.shape[1]
    
    #forword propagation
    Z = np.dot(W,X) + b
    A = sigmoid(Z)
    J = (-1/m)* np.sum(Y * np.log1p(A) + (1-Y) * (np.log1p(1-A)))
    
    #backword propagation
    dZ = A - Y
    dW = (1/m)*np.dot(dZ,X.T)
    db = (1/m)*np.sum(dZ)
    
    assert(dW.shape == W.shape)
    assert(db.dtype == float)
    J = np.squeeze(J)
    assert(J.shape == ())
    
    grad = {"dW":dW,
            "db":db}
    
    return grad,J

In [43]:
def optimize(W, b, X, Y, num_iterations, learning_rate):
    costs = []
    for i in range(num_iterations):
        
        grad,J = propagate(X,Y,W,b)
    
        dW = grad["dW"]
        db = grad["db"]
        
        W = W - learning_rate*dW
        b = b - learning_rate*db
        
        if i%100==0:
            costs.append(J)
            print ("Cost after iteration %i: %f" % (i, J))
        
    params = {"W": W,
              "b": b}
    grad = {"dW": dW,
             "db": db}
    
    return params, grad, costs

In [44]:
#predict function
def predict(W,b,X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    W = W.reshape(1,X.shape[0])
    Z = np.dot(W, X) + b
    A = sigmoid(Z)
    
    for i in range(A.shape[1]):
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
       
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [45]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5):
    
    # initialize parameters with zeros
    W, b = initialize(X_train.shape[0])

    # Gradient descent
    parameters, grads, costs = optimize(W, b, X_train, Y_train, num_iterations, learning_rate)
    
    # Retrieve parameters w and b from dictionary "parameters"
    W = parameters["W"]
    b = parameters["b"]
    
    # Predict test/train set examples
    Y_prediction_test = predict(W, b, X_test)
    Y_prediction_train = predict(W, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "W" : W, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
d = model(X_train, Y_train, X_test, Y_test, num_iterations = 5000, learning_rate = 0.0002)

In [ ]:
def plot_cost(costs):
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()